In [1]:
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy.engine import create_engine


In [2]:
LIMIT = None
#LIMIT = 5000
used_cols_with_target = ['item_id','category_name','price','deal_probability']
used_cols_without_target = ['item_id','category_name','price']
target = 'deal_probability'

Create a databas called avito.db and add train.csv into the database

In [3]:
# ONLY NEED TO DO ONCE TO CREATE DATABASE

#df = pd.read_csv('data/train.csv').dropna()
#avito = sqlite3.connect("avito.db")
#df.to_sql('data/train', avito)

#df = pd.read_csv('data/periods_train.csv').dropna()
#avito = sqlite3.connect("avito.db")
#df.to_sql('data/periods_train', avito)

In [4]:
train_pd = pd.read_csv('data/train.csv', usecols=used_cols_with_target)[:LIMIT]
train = train_pd.drop(target,axis = 1).dropna()
train_active = pd.read_csv('data/train_active.csv', usecols=used_cols_without_target)[:LIMIT].dropna()
train_all = train.append(train_active)


In [5]:
price_category_mean = train_all[['price', 'category_name']].groupby('category_name').mean().reset_index()
price_category_std = train_all[['price', 'category_name']].groupby('category_name').std().reset_index()
cate_train_active = pd.merge(price_category_mean,price_category_std, on='category_name')
cate_train_active.rename(columns={'price_x':'price_mean', 'price_y':'price_std'}, inplace=True)
cate_train_active

,category_name,price_mean,price_std
0,Автомобили,1.781997e+06,7.499982e+08
1,Аквариум,5.750065e+03,3.778824e+05
2,Аудио и видео,3.838495e+05,1.861367e+08
3,Билеты и путешествия,1.286506e+05,1.022621e+07
4,Бытовая техника,6.450468e+03,3.510457e+04
5,Велосипеды,1.090584e+04,3.247186e+05
6,Водный транспорт,6.294388e+05,1.015594e+07
7,Гаражи и машиноместа,3.006453e+05,3.562070e+06
8,Готовый бизнес,4.972508e+06,3.602614e+07
9,Грузовики и спецтехника,9.505564e+05,2.764812e+07


In [6]:
# assign category price mean and std values to each item
#price_category_mean = train_active[['price', 'category_name']].groupby('category_name').mean().reset_index()
#price_category_std = train_active[['price', 'category_name']].groupby('category_name').std().reset_index()

In [7]:
cate_train_active = cate_train_active[['category_name','price_mean','price_std']]

In [8]:
cate_train_active.to_csv('cate_price.csv', index=False)

In [9]:
#cate_train_active = pd.read_csv('cate_price')

Calculate price normal

In [10]:
#train = pd.read_csv('data/train.csv', usecols=used_cols)
#train = pd.read_csv('data/train.csv', usecols=used_cols).dropna()
train['category_name'].nunique()
train['price_normal'] = 0
train.head()

,item_id,category_name,price,price_normal
0,b912c3c6a6ad,Товары для детей и игрушки,400.0,0
1,2dac0150717d,Мебель и интерьер,3000.0,0
2,ba83aefab5dc,Аудио и видео,4000.0,0
3,02996f1dd2ea,Товары для детей и игрушки,2200.0,0
4,7c90be56d2ab,Автомобили,40000.0,0


In [11]:
cate_train_active = pd.read_csv('cate_price.csv')
cate_train_active.head()

,category_name,price_mean,price_std
0,Автомобили,1.781997e+06,7.499982e+08
1,Аквариум,5.750065e+03,3.778824e+05
2,Аудио и видео,3.838495e+05,1.861367e+08
3,Билеты и путешествия,1.286506e+05,1.022621e+07
4,Бытовая техника,6.450468e+03,3.510457e+04


In [12]:
# TEST IF TRAIN MAKES SENSE
#cate_train = train.groupby('category_name').agg({'price': [np.mean, np.std]})
#cate_train

In [13]:
train_pd['cat_price'] = np.nan
train_pd['cat_std'] = np.nan
train_pd['norm_price'] = np.nan

In [14]:
train_pd

,item_id,category_name,price,deal_probability,cat_price,cat_std,norm_price
0,b912c3c6a6ad,Товары для детей и игрушки,400.0,0.12789,NaN,NaN,NaN
1,2dac0150717d,Мебель и интерьер,3000.0,0.00000,NaN,NaN,NaN
2,ba83aefab5dc,Аудио и видео,4000.0,0.43177,NaN,NaN,NaN
3,02996f1dd2ea,Товары для детей и игрушки,2200.0,0.80323,NaN,NaN,NaN
4,7c90be56d2ab,Автомобили,40000.0,0.20797,NaN,NaN,NaN
5,51e0962387f7,Товары для детей и игрушки,1300.0,0.80323,NaN,NaN,NaN
6,c4f260a2b48a,Ремонт и строительство,11000.0,0.00000,NaN,NaN,NaN
7,6b71309d6a8a,"Одежда, обувь, аксессуары",500.0,0.80323,NaN,NaN,NaN
8,c5b969cb63a2,"Одежда, обувь, аксессуары",500.0,0.00000,NaN,NaN,NaN
9,b1570962e68c,Детская одежда и обувь,400.0,0.00000,NaN,NaN,NaN


In [15]:
for x,item in enumerate(train_pd['category_name'].unique()):    
    train_pd.cat_price[ train_pd.category_name == item ] =  cate_train_active[cate_train_active.category_name == item].price_mean.item()
    train_pd.cat_std[ train_pd.category_name == item ] =  cate_train_active[cate_train_active.category_name == item].price_std.item()

train_pd

/Users/zexuanxu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/zexuanxu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,item_id,category_name,price,deal_probability,cat_price,cat_std,norm_price
0,b912c3c6a6ad,Товары для детей и игрушки,400.0,0.12789,1.128727e+05,9.832614e+07,NaN
1,2dac0150717d,Мебель и интерьер,3000.0,0.00000,1.105329e+04,1.162178e+06,NaN
2,ba83aefab5dc,Аудио и видео,4000.0,0.43177,3.838495e+05,1.861367e+08,NaN
3,02996f1dd2ea,Товары для детей и игрушки,2200.0,0.80323,1.128727e+05,9.832614e+07,NaN
4,7c90be56d2ab,Автомобили,40000.0,0.20797,1.781997e+06,7.499982e+08,NaN
5,51e0962387f7,Товары для детей и игрушки,1300.0,0.80323,1.128727e+05,9.832614e+07,NaN
6,c4f260a2b48a,Ремонт и строительство,11000.0,0.00000,3.757785e+04,1.852780e+07,NaN
7,6b71309d6a8a,"Одежда, обувь, аксессуары",500.0,0.80323,7.378630e+03,3.268668e+06,NaN
8,c5b969cb63a2,"Одежда, обувь, аксессуары",500.0,0.00000,7.378630e+03,3.268668e+06,NaN
9,b1570962e68c,Детская одежда и обувь,400.0,0.00000,4.708419e+03,1.026324e+06,NaN


In [16]:
# SLOW ALGORITHM; Do not use
#for x in range (0,len(train_pd)-1):
#    train_pd.cat_price[x] =  cate_train_active.price_mean [ cate_train_active.category_name == train_pd.category_name[x] ]
#    train_pd.cat_std[x] =  cate_train_active.price_std [ cate_train_active.category_name == train_pd.category_name[x] ] 
#train_pd

In [17]:
train_pd

,item_id,category_name,price,deal_probability,cat_price,cat_std,norm_price
0,b912c3c6a6ad,Товары для детей и игрушки,400.0,0.12789,1.128727e+05,9.832614e+07,NaN
1,2dac0150717d,Мебель и интерьер,3000.0,0.00000,1.105329e+04,1.162178e+06,NaN
2,ba83aefab5dc,Аудио и видео,4000.0,0.43177,3.838495e+05,1.861367e+08,NaN
3,02996f1dd2ea,Товары для детей и игрушки,2200.0,0.80323,1.128727e+05,9.832614e+07,NaN
4,7c90be56d2ab,Автомобили,40000.0,0.20797,1.781997e+06,7.499982e+08,NaN
5,51e0962387f7,Товары для детей и игрушки,1300.0,0.80323,1.128727e+05,9.832614e+07,NaN
6,c4f260a2b48a,Ремонт и строительство,11000.0,0.00000,3.757785e+04,1.852780e+07,NaN
7,6b71309d6a8a,"Одежда, обувь, аксессуары",500.0,0.80323,7.378630e+03,3.268668e+06,NaN
8,c5b969cb63a2,"Одежда, обувь, аксессуары",500.0,0.00000,7.378630e+03,3.268668e+06,NaN
9,b1570962e68c,Детская одежда и обувь,400.0,0.00000,4.708419e+03,1.026324e+06,NaN


In [18]:
train_pd['norm_price'] =  (train_pd['price'] - train_pd['cat_price'] ) / train_pd['cat_std'] 

In [19]:
#for i, item in enumerate(train['item_id'][0:]):    
#    for j, cate in enumerate(cate_train_active['category_name']):
#        if train['category_name'][i] == cate:
#            train['price_normal'][i] = (train['price'][i] - cate_train_active['price_mean'][j])/cate_train_active['price_std'][j]             

In [20]:
train_pd

,item_id,category_name,price,deal_probability,cat_price,cat_std,norm_price
0,b912c3c6a6ad,Товары для детей и игрушки,400.0,0.12789,1.128727e+05,9.832614e+07,-0.001144
1,2dac0150717d,Мебель и интерьер,3000.0,0.00000,1.105329e+04,1.162178e+06,-0.006929
2,ba83aefab5dc,Аудио и видео,4000.0,0.43177,3.838495e+05,1.861367e+08,-0.002041
3,02996f1dd2ea,Товары для детей и игрушки,2200.0,0.80323,1.128727e+05,9.832614e+07,-0.001126
4,7c90be56d2ab,Автомобили,40000.0,0.20797,1.781997e+06,7.499982e+08,-0.002323
5,51e0962387f7,Товары для детей и игрушки,1300.0,0.80323,1.128727e+05,9.832614e+07,-0.001135
6,c4f260a2b48a,Ремонт и строительство,11000.0,0.00000,3.757785e+04,1.852780e+07,-0.001434
7,6b71309d6a8a,"Одежда, обувь, аксессуары",500.0,0.80323,7.378630e+03,3.268668e+06,-0.002104
8,c5b969cb63a2,"Одежда, обувь, аксессуары",500.0,0.00000,7.378630e+03,3.268668e+06,-0.002104
9,b1570962e68c,Детская одежда и обувь,400.0,0.00000,4.708419e+03,1.026324e+06,-0.004198


In [21]:
train_pd.to_csv('train_price_normalized.csv')

Select the item_id term only

In [22]:
#item = pd.read_sql('SELECT item_id,category_name,price FROM train_active WHERE price > 10000', avito)


SELECT all items with price > 10000

In [23]:
#price = pd.read_sql('SELECT * FROM train WHERE price > 10000 ', avito)
#price

In [24]:
test = pd.read_csv('data/test.csv', usecols=used_cols_without_target)[:LIMIT]

In [25]:
test['cat_price'] = np.nan
test['cat_std'] = np.nan
test['norm_price'] = np.nan

In [26]:
test

,item_id,category_name,price,cat_price,cat_std,norm_price
0,6544e41a8817,Детская одежда и обувь,NaN,NaN,NaN,NaN
1,65b9484d670f,Велосипеды,3000.0,NaN,NaN,NaN
2,8bab230b2ecd,Аудио и видео,15000.0,NaN,NaN,NaN
3,8e348601fefc,Бытовая техника,4500.0,NaN,NaN,NaN
4,8bd2fe400b89,Товары для детей и игрушки,4900.0,NaN,NaN,NaN
5,c63dbd6c657f,Ремонт и строительство,500.0,NaN,NaN,NaN
6,6d1a410df86e,Ноутбуки,20990.0,NaN,NaN,NaN
7,e8d3e7922b80,Телефоны,990.0,NaN,NaN,NaN
8,2bc1ab208462,Товары для компьютера,1200.0,NaN,NaN,NaN
9,7e05d77a9181,Детская одежда и обувь,400.0,NaN,NaN,NaN


In [27]:
cate_train_active

,category_name,price_mean,price_std
0,Автомобили,1.781997e+06,7.499982e+08
1,Аквариум,5.750065e+03,3.778824e+05
2,Аудио и видео,3.838495e+05,1.861367e+08
3,Билеты и путешествия,1.286506e+05,1.022621e+07
4,Бытовая техника,6.450468e+03,3.510457e+04
5,Велосипеды,1.090584e+04,3.247186e+05
6,Водный транспорт,6.294388e+05,1.015594e+07
7,Гаражи и машиноместа,3.006453e+05,3.562070e+06
8,Готовый бизнес,4.972508e+06,3.602614e+07
9,Грузовики и спецтехника,9.505564e+05,2.764812e+07


In [28]:
for x,item in enumerate(test['category_name'].unique()):    
    test.cat_price[ test.category_name == item ] =  cate_train_active[cate_train_active.category_name == item].price_mean.item()
    test.cat_std[ test.category_name == item ] =  cate_train_active[cate_train_active.category_name == item].price_std.item()


/Users/zexuanxu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/zexuanxu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
test['norm_price'] =  (test['price'] - test['cat_price'] ) / test['cat_std'] 

In [32]:
test

,item_id,category_name,price,cat_price,cat_std,norm_price
0,6544e41a8817,Детская одежда и обувь,NaN,4.708419e+03,1.026324e+06,NaN
1,65b9484d670f,Велосипеды,3000.0,1.090584e+04,3.247186e+05,-0.024347
2,8bab230b2ecd,Аудио и видео,15000.0,3.838495e+05,1.861367e+08,-0.001982
3,8e348601fefc,Бытовая техника,4500.0,6.450468e+03,3.510457e+04,-0.055562
4,8bd2fe400b89,Товары для детей и игрушки,4900.0,1.128727e+05,9.832614e+07,-0.001098
5,c63dbd6c657f,Ремонт и строительство,500.0,3.757785e+04,1.852780e+07,-0.002001
6,6d1a410df86e,Ноутбуки,20990.0,1.420191e+04,2.365165e+04,0.287003
7,e8d3e7922b80,Телефоны,990.0,7.635322e+05,4.596223e+08,-0.001659
8,2bc1ab208462,Товары для компьютера,1200.0,5.538548e+03,9.756872e+05,-0.004447
9,7e05d77a9181,Детская одежда и обувь,400.0,4.708419e+03,1.026324e+06,-0.004198


In [31]:
test.to_csv('test_price_normalized.csv')